In [ ]:
import pandas as pd

df = pd.read_csv('MSFT_data.csv')

df

In [ ]:
df = df[['Date', 'Close']]

df

In [ ]:
df['Date']

In [ ]:
import datetime
#spremenim stringe v stevilke
def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return datetime.datetime(year=year, month=month, day=day)

datetime_object = str_to_datetime('1986-03-19')
datetime_object

In [ ]:
df

In [ ]:
df['Date'] = df['Date'].apply(str_to_datetime)
df['Date']

In [ ]:
df.index = df.pop('Date')
df

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df.index, df['Close'])

In [ ]:
import numpy as np

def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
  first_date = str_to_datetime(first_date_str)
  last_date  = str_to_datetime(last_date_str)

  target_date = first_date

  dates = []
  X, Y = [], []

  last_time = False
  while True:
    df_subset = dataframe.loc[:target_date].tail(n+1)

    if len(df_subset) != n+1:
      print(f'Error: Window of size {n} is too large for date {target_date}')
      return

    values = df_subset['Close'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

    if last_time:
      break

    target_date = next_date

    if target_date == last_date:
      last_time = True

  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates

  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]

  ret_df['Target'] = Y

  return ret_df

# Start day second time around: '2021-03-25'
windowed_df = df_to_windowed_df(df,
                                '2021-03-25',
                                '2022-03-23',
                                n=3)
windowed_df

In [ ]:
def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
    first_date = str_to_datetime(first_date_str)
    last_date  = str_to_datetime(last_date_str)

    df = dataframe.sort_index()  # za vsak slučaj posortiramo po indeksu
    df = df.loc[first_date:last_date]

    dates = df.index[n:]  # datumi, za katere imamo n prejšnjih vrednosti
    X, Y = [], []

    for i in range(n, len(df)):
        window = df['Close'].iloc[i-n:i+1].to_numpy()
        if len(window) != n+1:
            continue  # varnostni ukrep
        x, y = window[:-1], window[-1]
        X.append(x)
        Y.append(y)

    ret_df = pd.DataFrame({'Target Date': dates})
    X = np.array(X)

    for i in range(n):
        ret_df[f'Target-{n-i}'] = X[:, i]
    ret_df['Target'] = Y

    return ret_df
windowed_df = df_to_windowed_df(df, '2023-03-25', '2025-03-23', n=3)
windowed_df


In [ ]:
def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  dates = df_as_np[:, 0]

  middle_matrix = df_as_np[:, 1:-1]
  X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

  Y = df_as_np[:, -1]

  return dates, X.astype(np.float32), Y.astype(np.float32)

dates, X, y = windowed_df_to_date_X_y(windowed_df)

dates.shape, X.shape, y.shape

In [ ]:
q_80 = int(len(dates) * .8)
q_90 = int(len(dates) * .9)

dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]

dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]

plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)

plt.legend(['Train', 'Validation', 'Test'])

In [ ]:
windowed_df.to_csv("windowed_data.csv", index=False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(1)])

model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.001),
              metrics=['mean_absolute_error'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)

In [ ]:
train_predictions = model.predict(X_train).flatten()

plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.legend(['Training Predictions', 'Training Observations'])

In [ ]:
# === Okno 1: LSTM → Conv1D → LSTM ===
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

model1 = Sequential([
    layers.Input(shape=(X_train.shape[1], X_train.shape[2])),  # (3,1)
    layers.LSTM(64, return_sequences=True),
    layers.Conv1D(32, kernel_size=2, activation='relu'),
    layers.LSTM(64),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model1.compile(
    loss='mse',
    optimizer=Adam(learning_rate=0.001),
    metrics=['mean_absolute_error']
)

# Oblikovanje vhodov za skalerje
X_train_reshaped = X_train.reshape(-1, 1)
X_val_reshaped = X_val.reshape(-1, 1)
X_test_reshaped = X_test.reshape(-1, 1)

# Inicializacija skalerjev
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fitiraj samo na treningu
scaler_X.fit(X_train_reshaped)
scaler_y.fit(y_train.reshape(-1, 1))

# Transformiraj vhodne in ciljne podatke
X_train_scaled = scaler_X.transform(X_train_reshaped).reshape(X_train.shape)
X_val_scaled   = scaler_X.transform(X_val_reshaped).reshape(X_val.shape)
X_test_scaled  = scaler_X.transform(X_test_reshaped).reshape(X_test.shape)

y_train_scaled = scaler_y.transform(y_train.reshape(-1, 1)).flatten()
y_val_scaled   = scaler_y.transform(y_val.reshape(-1, 1)).flatten()
y_test_scaled  = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

history1 = model1.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=100,
    verbose=2
)


# Napovedi na treining podatkih
train_pred_scaled = model1.predict(X_train_scaled).flatten()
train_predictions = scaler_y.inverse_transform(train_pred_scaled.reshape(-1, 1)).flatten()

plt.figure(figsize=(12, 4))
plt.plot(dates_train, train_predictions, label='Training Predictions')
plt.plot(dates_train, y_train, label='Training Observations')
plt.title('Training Set: Napovedi proti opazovanim vrednostim')
plt.legend()
plt.grid(True)
plt.show()

# Napovedi na validacijskih podatkih
val_pred_scaled = model1.predict(X_val_scaled).flatten()
val_predictions = scaler_y.inverse_transform(val_pred_scaled.reshape(-1, 1)).flatten()

plt.figure(figsize=(12, 4))
plt.plot(dates_val, val_predictions, label='Validation Predictions')
plt.plot(dates_val, y_val, label='Validation Observations')
plt.title('Validation Set: Napovedi proti opazovanim vrednostim')
plt.legend()
plt.grid(True)
plt.show()

# Napovedi na testnih podatkih
test_pred_scaled = model1.predict(X_test_scaled).flatten()
test_predictions = scaler_y.inverse_transform(test_pred_scaled.reshape(-1, 1)).flatten()

plt.figure(figsize=(12, 4))
plt.plot(dates_test, test_predictions, label='Testing Predictions')
plt.plot(dates_test, y_test, label='Testing Observations')
plt.title('Testing Set: Napovedi proti opazovanim vrednostim')
plt.legend()
plt.grid(True)
plt.show()

# Izris izgube učenja
plt.figure(figsize=(10, 4))
plt.plot(history1.history['loss'], label='Training Loss')
plt.plot(history1.history['val_loss'], label='Validation Loss')
plt.title('Loss med učenjem')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.grid(True)
plt.show()

print("Min y_train:", np.min(y_train))
print("Max y_train:", np.max(y_train))
print("Povprečje y_train:", np.mean(y_train))
print("Standardni odklon y_train:", np.std(y_train))

for i in range(10):
    print(f"Napoved: {train_predictions[i]:.2f} | Dejanska: {y_train[i]:.2f}")


In [ ]:
model1.summary()
# Vizualizacija izgube (loss) med učenjem
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(history1.history['loss'], label='Train Loss (MSE)')
plt.plot(history1.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Izguba med učenjem modela1 (LSTM → Conv1D → LSTM)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# === 1. Naloži model in skalerje ===

# === 2. Vhod: zadnje 3 vrednosti cene delnice (npr. realne vrednosti) ===
last_3_prices = np.array([261.12, 261.50, 266.82])  # nadomesti z aktualnimi podatki

# === 3. Preoblikuj in skaliraj vhodne podatke ===
X_input = last_3_prices.reshape(-1, 1)                         # v (3,1)
X_input_scaled = scaler_X.transform(X_input).reshape(1, 3, 1)  # v (1,3,1)

# === 4. Napoved in denormalizacija ===
y_pred_scaled = model1.predict(X_input_scaled)                 # skalirana napoved
y_pred = scaler_y.inverse_transform(y_pred_scaled)[0][0]      # realna vrednost

# === 5. Izpis ===
print(f"Napovedana cena delnice: {y_pred:.2f}")


In [ ]:
# === Okno 2: Čisti GRU in Hibrid LSTM → GRU s skaliranjem ===
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

# 1. NORMALIZACIJA podatkov
# --------------------------------
# Preoblikuj 3D X v 2D za MinMaxScaler
X_train_flat = X_train.reshape(-1, 1)
X_val_flat   = X_val.reshape(-1, 1)
X_test_flat  = X_test.reshape(-1, 1)

# Inicializiraj skalerje
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Prilagodi (fit) samo na treningu
scaler_X.fit(X_train_flat)
scaler_y.fit(y_train.reshape(-1, 1))

# Pretvori podatke nazaj v 3D
X_train_scaled = scaler_X.transform(X_train_flat).reshape(X_train.shape)
X_val_scaled   = scaler_X.transform(X_val_flat).reshape(X_val.shape)
X_test_scaled  = scaler_X.transform(X_test_flat).reshape(X_test.shape)

# Skaliraj y vrednosti
y_train_scaled = scaler_y.transform(y_train.reshape(-1, 1)).flatten()
y_val_scaled   = scaler_y.transform(y_val.reshape(-1, 1)).flatten()
y_test_scaled  = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

# 2. DEFINICIJA in UČENJE MODELOV
# --------------------------------
# 2a) Čisti GRU
model2a = Sequential([
    layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    layers.GRU(64),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model2a.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])
history2a = model2a.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=100, verbose=2
)

# 2b) Hibrid LSTM → GRU
model2b = Sequential([
    layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    layers.LSTM(64, return_sequences=True),
    layers.GRU(64),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model2b.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])
history2b = model2b.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=100, verbose=2
)

# 3. NAPOVEDI in DENORMALIZACIJA
# --------------------------------
# Napovedi (skalirane) na testnem sklopu
pred2a_scaled = model2a.predict(X_test_scaled).flatten()
pred2b_scaled = model2b.predict(X_test_scaled).flatten()

# Pretvori nazaj v originalne vrednosti
pred2a = scaler_y.inverse_transform(pred2a_scaled.reshape(-1,1)).flatten()
pred2b = scaler_y.inverse_transform(pred2b_scaled.reshape(-1,1)).flatten()

# 4. GRAFIČNI PRIKAZ
# --------------------------------
plt.figure(figsize=(12, 5))
plt.plot(dates_test, y_test,       label='Prave vrednosti', color='black', linewidth=2)
plt.plot(dates_test, pred2a,       label='Model2a: Čisti GRU', linestyle='--')
plt.plot(dates_test, pred2b,       label='Model2b: LSTM → GRU', linestyle=':')
plt.title('Primerjava napovedi model2a in model2b na testnem sklopu')
plt.xlabel('Datum')
plt.ylabel('Cena delnice')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# === 1. Naloži skalerje in modela (če niso že v okolju) ===
# Predpostavimo, da imaš to že v RAM-u:
# - model2a, model2b
# - scaler_X, scaler_y

# Če jih nalagaš iz datotek:
# model2a = load_model('model2a.h5')
# model2b = load_model('model2b.h5')
# scaler_X = joblib.load('scaler_X.pkl')
# scaler_y = joblib.load('scaler_y.pkl')

# === 2. Vhod: zadnje 3 realne cene delnice ===
last_3_prices = np.array([261.12, 261.50, 266.82])  # nadomesti s svojimi podatki

# === 3. Skaliraj vhod ===
X_input = last_3_prices.reshape(-1, 1)                         # (3,1)
X_input_scaled = scaler_X.transform(X_input).reshape(1, 3, 1)  # (1,3,1)

# === 4. Napoved za oba modela ===
y_pred2a_scaled = model2a.predict(X_input_scaled)
y_pred2b_scaled = model2b.predict(X_input_scaled)

# === 5. Denormalizacija izhoda ===
y_pred2a = scaler_y.inverse_transform(y_pred2a_scaled)[0][0]
y_pred2b = scaler_y.inverse_transform(y_pred2b_scaled)[0][0]

# === 6. Izpis rezultatov ===
print(f"Model2a (Čisti GRU) napoved:        {y_pred2a:.2f}")
print(f"Model2b (LSTM → GRU) napoved:       {y_pred2b:.2f}")


In [ ]:
import pandas as pd
from tensorflow.keras.models import clone_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

months = pd.to_datetime(dates).to_period('M')
unique_months = sorted(months.unique())

results = []

for i in range(len(unique_months) - 1):
    train_month = unique_months[i]
    test_month  = unique_months[i + 1]

    # Indeksi za train in test
    train_idx = months <= train_month
    test_idx  = months == test_month

    X_tr, y_tr = X[train_idx], y[train_idx]
    X_te, y_te = X[test_idx], y[test_idx]

    # --- NOVO: fitiraj scaler samo na X_tr, y_tr ---
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_tr_flat = X_tr.reshape(-1, 1)
    X_te_flat = X_te.reshape(-1, 1)

    scaler_X.fit(X_tr_flat)
    scaler_y.fit(y_tr.reshape(-1, 1))

    X_tr_scaled = scaler_X.transform(X_tr_flat).reshape(X_tr.shape)
    X_te_scaled = scaler_X.transform(X_te_flat).reshape(X_te.shape)

    y_tr_scaled = scaler_y.transform(y_tr.reshape(-1, 1)).flatten()
    y_te_scaled = scaler_y.transform(y_te.reshape(-1, 1)).flatten()

    # Kloniraj in inicializiraj model iz nič (bolj resnično rolling)
    model_rolling = clone_model(model1)
    model_rolling.compile(
        loss='mse',
        optimizer=Adam(learning_rate=0.001),
        metrics=['mean_absolute_error']
    )

    # Treniraj na skaliranih podatkih
    model_rolling.fit(
        X_tr_scaled, y_tr_scaled,
        epochs=20,
        verbose=0
    )

    # Evaluiraj na skaliranih testnih podatkih
    loss, mae_scaled = model_rolling.evaluate(X_te_scaled, y_te_scaled, verbose=0)

    # Denormaliziraj MAE ali napoved, če želiš metriko v originalni enoti:
    # pred_te_scaled = model_rolling.predict(X_te_scaled)
    # pred_te = scaler_y.inverse_transform(pred_te_scaled)
    # mae = np.mean(np.abs(pred_te.flatten() - y_te))

    print(f"Train {train_month} → Test {test_month}: MAE_scaled = {mae_scaled:.4f}")
    results.append({
        'train_month': str(train_month),
        'test_month' : str(test_month),
        'mae_scaled' : mae_scaled
    })
# Po koncu zanke:
model_rolling_final = model_rolling

# 1) Napovedi na train množici
train_rolling_pred_scaled = model_rolling_final.predict(X_train_scaled).flatten()
train_rolling_pred = scaler_y.inverse_transform(train_rolling_pred_scaled.reshape(-1,1)).flatten()

plt.figure(figsize=(12,4))
plt.plot(dates_train, train_rolling_pred, label='Rolling Model Predictions')
plt.plot(dates_train, y_train,           label='Training Observations')
plt.title('Rolling Model – Training Set')
plt.legend()
plt.grid(True)
plt.show()

# 2) Napovedi na validation množici
val_rolling_pred_scaled = model_rolling_final.predict(X_val_scaled).flatten()
val_rolling_pred = scaler_y.inverse_transform(val_rolling_pred_scaled.reshape(-1,1)).flatten()

plt.figure(figsize=(12,4))
plt.plot(dates_val, val_rolling_pred, label='Rolling Model Predictions')
plt.plot(dates_val, y_val,           label='Validation Observations')
plt.title('Rolling Model – Validation Set')
plt.legend()
plt.grid(True)
plt.show()

# 3) Napovedi na test množici
test_rolling_pred_scaled = model_rolling_final.predict(X_test_scaled).flatten()
test_rolling_pred = scaler_y.inverse_transform(test_rolling_pred_scaled.reshape(-1,1)).flatten()

plt.figure(figsize=(12,4))
plt.plot(dates_test, test_rolling_pred, label='Rolling Model Predictions')
plt.plot(dates_test, y_test,           label='Testing Observations')
plt.title('Rolling Model – Test Set')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import joblib

# Shrani scalerje
joblib.dump(scaler_X, "scaler_X.gz")
joblib.dump(scaler_y, "scaler_y.gz")
# Shrani modela v datoteki
model2a.save("model2a_gru.h5")
model2b.save("model2b_lstm_gru.h5")


In [ ]:
import os
import joblib

# 1. Ustvari mapo
output_dir = "shrani_model"
os.makedirs(output_dir, exist_ok=True)

# 2. Shrani scalerje v mapo
joblib.dump(scaler_X, os.path.join(output_dir, "scaler_X.gz"))
joblib.dump(scaler_y, os.path.join(output_dir, "scaler_y.gz"))

# 3. Shrani modela v mapo
model2a.save(os.path.join(output_dir, "model2a_gru.h5"))
model2b.save(os.path.join(output_dir, "model2b_lstm_gru.h5"))

# 4. (Neobvezno) Stisni mapo za prenos


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras import losses
import joblib
import numpy as np

# === 1. Branje modelov ===
model2a = load_model(
    'model2a_gru.h5',
    custom_objects={'mse': losses.MeanSquaredError()}
)
model2b = load_model(
    'model2b_lstm_gru.h5',
    custom_objects={'mse': losses.MeanSquaredError()}
)
scaler_X = joblib.load('scaler_X.gz')
scaler_y = joblib.load('scaler_y.gz')

# === 2. Vhod: zadnje 3 realne cene delnice ===
last_3_prices = np.array([261.12, 261.50, 266.82])

# === 3. Priprava in skaliranje ===
X_input = last_3_prices.reshape(-1, 1)
X_input_scaled = scaler_X.transform(X_input).reshape(1, 3, 1)

# === 4. Napovedi ===
y_pred2a = scaler_y.inverse_transform(model2a.predict(X_input_scaled))[0][0]
y_pred2b = scaler_y.inverse_transform(model2b.predict(X_input_scaled))[0][0]

# === 5. Izpis rezultatov ===
print(f"Napoved (GRU model):        {y_pred2a:.2f}")
print(f"Napoved (LSTM → GRU model): {y_pred2b:.2f}")


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras import losses
import joblib
import numpy as np
import os

# === 0. Pot do mape ===
model_dir = "shrani_model"

# === 1. Branje modelov ===
model2a = load_model(
    os.path.join(model_dir, 'model2a_gru.h5'),
    custom_objects={'mse': losses.MeanSquaredError()}
)
model2b = load_model(
    os.path.join(model_dir, 'model2b_lstm_gru.h5'),
    custom_objects={'mse': losses.MeanSquaredError()}
)
scaler_X = joblib.load(os.path.join(model_dir, 'scaler_X.gz'))
scaler_y = joblib.load(os.path.join(model_dir, 'scaler_y.gz'))

# === 2. Vhod: zadnje 3 realne cene delnice ===
last_3_prices = np.array([261.12, 261.50, 266.82])

# === 3. Priprava in skaliranje ===
X_input = last_3_prices.reshape(-1, 1)
X_input_scaled = scaler_X.transform(X_input).reshape(1, 3, 1)

# === 4. Napovedi ===
y_pred2a = scaler_y.inverse_transform(model2a.predict(X_input_scaled))[0][0]
y_pred2b = scaler_y.inverse_transform(model2b.predict(X_input_scaled))[0][0]

# === 5. Izpis rezultatov ===
print(f"Napoved (GRU model):        {y_pred2a:.2f}")
print(f"Napoved (LSTM → GRU model): {y_pred2b:.2f}")


In [ ]:
val_predictions = model.predict(X_val).flatten()

plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.legend(['Validation Predictions', 'Validation Observations'])

In [ ]:
test_predictions = model.predict(X_test).flatten()

plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Testing Predictions', 'Testing Observations'])

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Napovedi obeh modelov
test_predictions_model = model.predict(X_test).flatten()
#test_predictions_model1 = model1.predict(X_test).flatten()
#test_predictions_model2a  = model2a.predict(X_test).flatten()
#test_predictions_model2b  = model2b.predict(X_test).flatten()
# Napoved rolling modela (zadnji iz rolling treninga)
#test_predictions_rolling = model_rolling.predict(X_test).flatten()

# === Napoved iz modela1 (skalirana) ===
test_pred_model1_scaled = model1.predict(X_test_scaled).flatten()  # Uporabi skaliran vhod!
test_predictions_model1 = scaler_y.inverse_transform(test_pred_model1_scaled.reshape(-1, 1)).flatten()
# 3) Model2a (Čisti GRU), zdaj na skaliranih
test_pred2a_scaled = model2a.predict(X_test_scaled).flatten()
test_predictions_model2a = scaler_y.inverse_transform(test_pred2a_scaled.reshape(-1, 1)).flatten()
# 4) Model2b (LSTM → GRU), prav tako na skaliranih
test_pred2b_scaled = model2b.predict(X_test_scaled).flatten()
test_predictions_model2b = scaler_y.inverse_transform(test_pred2b_scaled.reshape(-1, 1)).flatten()

# 5) Rolling model (treniran na skaliranih podatkih)
test_pred_rolling_scaled = model_rolling.predict(X_test_scaled).flatten()
test_predictions_rolling = scaler_y.inverse_transform(test_pred_rolling_scaled.reshape(-1, 1)).flatten()
# Risanje obeh napovedi + dejanske vrednosti
plt.figure(figsize=(14, 7))

# Prava vrednost
plt.plot(dates_test, y_test, label='True Values', color='black', linewidth=2)

plt.plot(dates_test, test_predictions_model,   label='Model (LSTM enostaven)', linestyle='--')
plt.plot(dates_test, test_predictions_model1,  label='Model1 (LSTM → Conv1D → LSTM)', linestyle='-.')
plt.plot(dates_test, test_predictions_model2a, label='Model2a (Čisti GRU)', linestyle=':')
plt.plot(dates_test, test_predictions_model2b, label='Model2b (LSTM → GRU)', linestyle='solid', alpha=0.8)
plt.plot(dates_test, test_predictions_rolling, label='Model Rolling (mesečni trening)', linestyle='-', linewidth=2, alpha=0.7)

plt.legend()
plt.title('Primerjava napovedi vseh modelov na testnem sklopu')
plt.xlabel('Datum')
plt.ylabel('Ciljna vrednost')
plt.grid(True)
plt.tight_layout()
plt.show()

# Pretvori v numpy array, če še ni
y_true = np.array(y_test)

# 1) Enostaven LSTM (model)
y_pred_model    = np.array(test_predictions_model)
# 2) model1
y_pred_model1   = np.array(test_predictions_model1)
# 3) model2a
y_pred_model2a  = np.array(test_predictions_model2a)
# 4) model2b
y_pred_model2b  = np.array(test_predictions_model2b)
# 5) rolling model
y_pred_rolling  = np.array(test_predictions_rolling)

# Funkcija za izpis metrike
def print_metrics(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    print(f"{name:30s} MAE = {mae:.4f}, MSE = {mse:.4f}")
    return mae

# Izračunaj za vse
mae_model    = print_metrics('Model (LSTM enostaven)',       y_true, y_pred_model)
mae_model1   = print_metrics('Model1 (LSTM→Conv1D→LSTM)',    y_true, y_pred_model1)
mae_model2a  = print_metrics('Model2a (Čisti GRU)',          y_true, y_pred_model2a)
mae_model2b  = print_metrics('Model2b (LSTM→GRU)',          y_true, y_pred_model2b)
mae_rolling  = print_metrics('Model Rolling (mesečni trening)', y_true, y_pred_rolling)

# Kateri model ima najmanjši MAE?
mae_values = {
    'Model (LSTM enostaven)'       : mae_model,
    'Model1 (LSTM→Conv1D→LSTM)'    : mae_model1,
    'Model2a (Čisti GRU)'          : mae_model2a,
    'Model2b (LSTM→GRU)'           : mae_model2b,
    'Model Rolling (mesečni trening)': mae_rolling
}

best_model = min(mae_values, key=mae_values.get)
print(f"\nNajboljši model po MAE je: {best_model} (MAE = {mae_values[best_model]:.4f})")

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score
)

def directional_accuracy(y_true, y_pred):
    # Izračuna, ali se spodnja napoved ujema s pravilno smerjo gibanja
    # (True = narašča, False = pada oz. enako)
    true_dir = np.sign(np.diff(y_true))
    pred_dir = np.sign(np.diff(y_pred))
    # Primerjamo samo tam, kjer imamo oboje (dolžina je len(y)-1)
    return np.mean(true_dir == pred_dir)

# Pripravimo slovar napovedi
preds = {
    'LSTM simple':    test_predictions_model,
    'LSTM→Conv1D→LSTM':  test_predictions_model1,
    'GRU':         test_predictions_model2a,
    'LSTM→GRU':          test_predictions_model2b,
    'Rolling model':     test_predictions_rolling
}

results = []
for name, y_pred in preds.items():
    y_true = y_test

    # Regresijske metrike
    mae  = mean_absolute_error(y_true, y_pred)
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # Directional Accuracy
    dir_acc = directional_accuracy(y_true, y_pred) * 100  # v %

    results.append({
        'Model':             name,
        'MAE':               round(mae, 4),
        'MSE':               round(mse, 4),
        'RMSE':              round(rmse, 4),
        'MAPE(%)':           round(mape, 2),
        'R2':                round(r2, 4),
        'Dir_Accuracy(%)':   round(dir_acc, 2)
    })

# Ustvarimo DataFrame
eval_df = pd.DataFrame(results)

# Shrani (append) v CSV
out_path = 'evaluation-results.csv'
eval_df.to_csv(
    out_path,
    mode='a',
    encoding='utf-8-sig',
    header=not os.path.exists(out_path),
    index=False
)

print("Rezultati metrike so shranjeni v", out_path)
print(eval_df)



In [ ]:
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Training Predictions',
            'Training Observations',
            'Validation Predictions',
            'Validation Observations',
            'Testing Predictions',
            'Testing Observations'])

In [ ]:
from copy import deepcopy

recursive_predictions = []
recursive_dates = np.concatenate([dates_val, dates_test])

for target_date in recursive_dates:
  last_window = deepcopy(X_train[-1])
  next_prediction = model.predict(np.array([last_window])).flatten()
  recursive_predictions.append(next_prediction)
  last_window[-1] = next_prediction

In [ ]:
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.plot(recursive_dates, recursive_predictions)
plt.legend(['Training Predictions',
            'Training Observations',
            'Validation Predictions',
            'Validation Observations',
            'Testing Predictions',
            'Testing Observations',
            'Recursive Predictions'])

In [ ]:
import numpy as np
#may 18,2022
#prava cena 254.08
# Zadnje tri cene delnice (vstavi dejanske vrednosti)
last_3_values = np.array([261.12, 261.50, 266.82])  # Te vrednosti so samo primer

# Preoblikujemo podatke v obliko, ki jo model pričakuje: (1, 3, 1)
# To pomeni: 1 primer, 3 mesečne cene, 1 značilnost (cena delnice)
last_3_values = last_3_values.reshape((1, 3, 1))

# Napoved prihodnje cene (napovedujemo naslednji mesec)
predicted_value = model.predict(last_3_values)

# Izpišemo napovedano ceno
print(f"Napovedana cena delnice v naslednjem mesecu je: {predicted_value[0][0]}")


In [ ]:
import numpy as np

# Vhodni podatki - zadnje 3 cene delnice
last_3_values = np.array([261.12, 261.50, 266.82]).reshape((1, 3, 1))

# Seznam modelov z imeni (da lepše izpišeš)
models = {
    "Model (enostaven LSTM)": model,
    "Model1 (LSTM → Conv1D → LSTM)": model1,
    "Model2a (Čisti GRU)": model2a,
    "Model2b (LSTM → GRU)": model2b,
    # Če imaš tudi rolling model:
    "Model Rolling (mesečni trening)": model_rolling
}

# Napoved za vsak model in izpis
for name, mdl in models.items():
    pred = mdl.predict(last_3_values)
    print(f"{name} napoved: {pred[0][0]:.2f}")


In [ ]:
import numpy as np

# Vhodni podatki - zadnje 3 cene delnice
last_3_values = np.array([231.12, 231.50, 236.82]).reshape((1, 3, 1))

# Seznam modelov z imeni (da lepše izpišeš)
models = {
    "Model (enostaven LSTM)": model,
    "Model1 (LSTM → Conv1D → LSTM)": model1,
    "Model2a (Čisti GRU)": model2a,
    "Model2b (LSTM → GRU)": model2b,
    # Če imaš tudi rolling model:
    "Model Rolling (mesečni trening)": model_rolling
}

# Napoved za vsak model in izpis
for name, mdl in models.items():
    pred = mdl.predict(last_3_values)
    print(f"{name} napoved: {pred[0][0]:.2f}")


In [ ]:
import numpy as np

# Zadnje 3 znane cene delnice (prilagodi po potrebi)
last_3_values = np.array([231.12, 231.50, 236.82]).reshape((1, 3, 1))

# Seznam modelov
models = {
    "Model (LSTM enostaven)": model,
    "Model1 (LSTM → Conv1D → LSTM)": model1,
    "Model2a (Čisti GRU)": model2a,
    "Model2b (LSTM → GRU)": model2b,
    "Model Rolling (mesečni trening)": model_rolling
}

# Modeli, ki uporabljajo skalirano y
scaled_models = {
    "Model1 (LSTM → Conv1D → LSTM)",
    "Model2a (Čisti GRU)",
    "Model2b (LSTM → GRU)",
    "Model Rolling (mesečni trening)"
}

# Napovedi
print("Napovedi za naslednji časovni korak:\n")
for name, mdl in models.items():
    pred = mdl.predict(last_3_values)
    if name in scaled_models:
        denorm_pred = scaler_y.inverse_transform(pred)[0][0]
    else:
        denorm_pred = pred[0][0]
    print(f"{name:<35} {denorm_pred:.2f}")
